In [ ]:
import os
%pwd
os.chdir('../')

In [2]:
%pwd

'c:\\Project\\MLops_basics_project'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [4]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories, save_json

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN
        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/mayank61/MLops_basics_project.mlflow",
           
        )

        return model_evaluation_config

In [10]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
import dagshub
dagshub.init(repo_owner='mayank61', repo_name='MLops_basics_project', mlflow=True)

import mlflow


[2025-08-12 22:27:38,489: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as mayank61

[2025-08-12 22:27:38,499: INFO: helpers: Accessing as mayank61]
[2025-08-12 22:27:39,397: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/mayank61/MLops_basics_project "HTTP/1.1 200 OK"]
[2025-08-12 22:27:40,182: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "mayank61/MLops_basics_project"

[2025-08-12 22:27:40,186: INFO: helpers: Initialized MLflow to track repo "mayank61/MLops_basics_project"]


Repository mayank61/MLops_basics_project initialized!

[2025-08-12 22:27:40,190: INFO: helpers: Repository mayank61/MLops_basics_project initialized!]


In [15]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        mlflow.sklearn.autolog()

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            

    



In [16]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-08-12 22:30:53,731: INFO: common: yaml file: C:\Project\MLops_basics_project\config\config.yaml loaded successfully]
[2025-08-12 22:30:53,734: INFO: common: yaml file: C:\Project\MLops_basics_project\params.yaml loaded successfully]
[2025-08-12 22:30:53,737: INFO: common: yaml file: C:\Project\MLops_basics_project\schema.yaml loaded successfully]
[2025-08-12 22:30:53,739: INFO: common: created directory at: artifacts]
[2025-08-12 22:30:53,741: INFO: common: created directory at: artifacts/model_evaluation]
[2025-08-12 22:30:55,967: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
🏃 View run masked-grub-807 at: https://dagshub.com/mayank61/MLops_basics_project.mlflow/#/experiments/0/runs/eb877c0a7a2b4182a85094c6f7368728
🧪 View experiment at: https://dagshub.com/mayank61/MLops_basics_project.mlflow/#/experiments/0
